In [81]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [22]:
books = pd.read_csv("BX-Books.csv", sep=';',encoding="latin-1",error_bad_lines=False)
users = pd.read_csv("BX-Users.csv", sep=';',encoding="latin-1",error_bad_lines=False)
rating = pd.read_csv("BX-Book-Ratings.csv", sep=';',encoding="latin-1",error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Harpragaas Singh\Anaconda3\lib\site-packages\IPython\core\interac

In [23]:
#PREPROCESSING DATA
#Books
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Book-Title':'Title', 'Book-Author':'Author', 'Year-Of-Publication':'Year', 'Publisher':'Publisher'}, inplace=True)
books.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [24]:
#Users
users.rename(columns = {'User-ID':'User_id', 'Location':'Location', 'Age':'Age'}, inplace=True)
users.head()

,User_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [25]:
#Ratings
rating.rename(columns = {'User-ID':'User_id', 'Book-Rating':'Rating'}, inplace=True)
rating.head()

,User_id,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [26]:
books.info()
print('\n')
users.info()
print('\n')
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
ISBN         271360 non-null object
Title        271360 non-null object
Author       271359 non-null object
Year         271360 non-null object
Publisher    271358 non-null object
dtypes: object(5)
memory usage: 10.4+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User_id     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User_id    1149780 non-null int64
ISBN       1149780 non-null object
Rating     1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


The above dataset is reliable with 271360 books data; with 278858 registered users and they have given about 1150000 rating, 
However, using the dataset could be a problem since it may contain users who have read just one or two books or just registerd on the website.

In order to solve that problem I will choose the user who has rated atleast 200 books and we will choose only thoese books which has recieved atleast 100 ratings.

In [85]:
#Extract Users and ratings of more than 200
x = rating['User_id'].value_counts()>200
y = x[x].index #user_ids
print(y.shape)
rating = rating[rating['User_id'].isin(y)]

(899,)


In [88]:
#Merging ratings and books
rating_with_books = rating.merge(books, on = 'ISBN')
rating_with_books.head()

,User_id,ISBN,Rating,Title,Author,Year,Publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [89]:
#Extract books that recieved more than 100 ratings
no_rating = rating_with_books.groupby('Title')['Rating'].count().reset_index()
no_rating.rename(columns={'Rating':'Number of Rating'},inplace = True)

final_rating= rating_with_books.merge(no_rating, on='Title')
final_rating.shape
final_rating = final_rating[final_rating['Number of Rating']>= 50]

#Drop duplicates values because if a user a=has rated the same book multiple time it will create a problem
final_rating.drop_duplicates(['User_id','Title'],inplace=True)

In [90]:
# Creating the final table
books_final = final_rating.pivot_table(columns='User_id',index='Title',values='Rating')
books_final.fillna(0,inplace=True)
books_final.head()

User_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


We have prepared the dataset but however there are alot of zero values and on clustering this could be a problem, thus, converting the pivot table to the sparse model and feed it to the model

In [91]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(books_final)

In [97]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric = "cosine", algorithm='auto')
model.fit(book_sparse)

NearestNeighbors(metric='cosine')

In [96]:
bookName = input("Enter a book name:")
number = int(input("Enter number of books to recommend: "))

Enter a book name:Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Enter number of books to recommend: 5


In [98]:
distances, suggestions = model.kneighbors(books_final.loc[bookName].values.reshape(1,-1), n_neighbors = number +1)
print("\n Recommended books are: \n")
for i in range(0, len(distances.flatten())):
    if i > 0:
        print(books_final.index[suggestions.flatten()[i]]) 


 Recommended books are: 

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
A Time to Kill
